In [1]:
import pandas as pd
import numpy as np
import random
import os

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")


In [2]:
# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [3]:
train = pd.read_csv('./train.csv')
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [ ]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)   
    
    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    ## 표준화 + 모델 진행
    
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'][train_close['종목코드']==code] = train_scaled

    # 변수의 가중치 화
    tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    final_return = np.mean(predictions)
    
    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

In [ ]:
###########################3 0725 ver 2

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()
    
    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    ## 표준화 + 모델 진행
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close['5_10평균']

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'] = train_scaled

    # 변수의 가중치 화
    #tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 수익률 계산
    final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]

    # 최종 누적 수익률
    

    # 최종 데이터프레임
    final_return = np.mean(train_close['5_10평균'][train_close['종목코드']==code])
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

In [ ]:
# 순위로 정리

#pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:.2f}'.format

final_df['final_return'] = final_df['final_return'].fillna(0)
final_df['순위'] = final_df['final_return'].rank(method='first', ascending=False).astype('int') # 각 순위를 중복없이 생성
pre=final_df.sort_values(by='순위')
final=pre[['종목코드','순위']]
final.head(10)

### 이동평균선 _ 20일선 기준 정배열 역배열 영향력 향샹 변수

In [ ]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    #train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)   
    
    #####---------- 변수 1 _ 5평균 - 10평균

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    #####---------- 변수 2 _ 정배열 / 역배열 영향력 강화

    # 정배열 역배열 개수 column 생성 및 종목별 20일간 정배열 역배열 개수 확인
    train_close['정배열'] = 0
    train_close['역배열'] = 0

    # 간격 차이 조합만 남아있는 데이터 프레임
    train_close_re = train_close[['종목코드', '종가', '5평균', '10평균', '20평균', '60평균', '정배열', '역배열']]
    train_close_re.reset_index(drop=False,inplace=True)


    ## 표준화 + 모델 진행
    
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'][train_close['종목코드']==code] = train_scaled

    # 변수의 가중치 화
    tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    final_return = np.mean(predictions)
    
    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

In [ ]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    #train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 등락폭
    #train_close["shift"] = train_close["종가"].shift(1)
    #train_close["등락폭"] = (train_close["종가"] - train_close["shift"])
    #train_close["등락비율"] = ((train_close["종가"] - train_close["shift"])/train_close["shift"])
    #train_close.drop("shift", axis=1,inplace=True)   
    
    #####---------- 변수 1 _ 5평균 - 10평균

    # 추세선 내부 간격 함수
    # 1 _ 5평균 - 10평균
    train_close['5_10평균'] = train_close["5평균"] - train_close["10평균"]

    # 2 _ 5평균 - 20평균
    #train_close['5_20평균'] = train_close["5평균"]-train_close["20평균"]
    
    # 3 _ 5평균 - 60평균
    #train_close['5_60평균'] = train_close["5평균"]-train_close["60평균"]
    
    # 4 _ 10평균 - 20평균
    #train_close['10_20평균'] = train_close["10평균"]-train_close["20평균"]

    # 5 _ 10평균 - 60평균
    #train_close['10_60평균'] = train_close["10평균"]-train_close["60평균"]

    # 6 _ 20평균 - 60평균
    #train_close['20_60평균'] = train_close["20평균"]-train_close["60평균"]

    #####---------- 변수 2 _ 정배열 / 역배열 영향력 강화

    # 정배열 역배열 개수 column 생성 및 종목별 20일간 정배열 역배열 개수 확인
    train_close['정배열'] = 0
    train_close['역배열'] = 0

    # 간격 차이 조합만 남아있는 데이터 프레임
    train_close_re = train_close[['종목코드', '종가', '5평균', '10평균', '20평균', '60평균', '정배열', '역배열']]
    train_close_re.reset_index(drop=False,inplace=True)


    ## 표준화 + 모델 진행
    
    # StandardScaler객체 생성
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()

    #scaler용 input 생성
    input = train_close[['5_10평균']]

    # StandardScaler 로 데이터 셋 변환. fit( ) 과 transform( ) 호출.  
    scaler.fit(input)
    train_scaled = scaler.transform(input)
    train_close['5_10평균'][train_close['종목코드']==code] = train_scaled

    # 변수의 가중치 화
    tc = train_close['5_10평균']

    # 모델 선언, 학습 및 추론
    model = ARIMA(tc, order=(2, 1, 2))
    model_fit = model.fit()
    predictions = model_fit.forecast(steps=15) # 향후 15개의 거래일에 대해서 예측

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    final_return = np.mean(predictions)
    
    # 최종 데이터프레임
    final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)


final_df

In [5]:
#### 0725 ver --- 이거로 진행

# 저장할 데이터 프레임
final_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    #####---------- 변수 2 _ 정배열 / 역배열 영향력 강화
    num = len(train_close["20평균"].index)
    cnt_plus=0
    cnt_minus=0
    start = train_close["20평균"].index + 
    for i in train_close["20평균"].index:
        for j in range(len(i)) 

        elif
        


    # 정배열 역배열 개수 column 생성 및 종목별 20일간 정배열 역배열 개수 확인
    train_close['정배열'] = 0
    train_close['역배열'] = 0

    # 간격 차이 조합만 남아있는 데이터 프레임
    #train_close_re = train_close[['종목코드', '종가', '5평균', '10평균', '20평균', '60평균', '정배열', '역배열']]
    #train_close_re.reset_index(drop=False,inplace=True)

    # 최종 예측치 평균
    #final_return = (predictions.iloc[-1] - predictions.iloc[0]) / predictions.iloc[0]
    #final_return = np.mean(predictions)
    
    # 최종 데이터프레임
    #final_df = final_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

train_close[train_close['종가'].rolling(window=5)].index()

100%|██████████| 2000/2000 [01:48<00:00, 18.37it/s]


TypeError: unhashable type: 'Series'

In [1]:
import pandas as pd
import numpy as np
import random
import os
import datetime as dt

from tqdm import tqdm
from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings("ignore")

# 시드 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pd.read_csv('./train.csv')
train

,일자,종목코드,종목명,거래량,시가,고가,저가,종가
0,20210601,A060310,3S,166690,2890,2970,2885,2920
1,20210601,A095570,AJ네트웍스,63836,5860,5940,5750,5780
2,20210601,A006840,AK홀딩스,103691,35500,35600,34150,34400
3,20210601,A054620,APS,462544,14600,14950,13800,14950
4,20210601,A265520,AP시스템,131987,29150,29150,28800,29050
...,...,...,...,...,...,...,...,...
987995,20230530,A189980,흥국에프엔비,272284,3005,3035,2955,2980
987996,20230530,A000540,흥국화재,50218,3250,3255,3195,3215
987997,20230530,A003280,흥아해운,130664,1344,1395,1340,1370
987998,20230530,A037440,희림,141932,9170,9260,9170,9200


In [6]:
# 저장할 데이터 프레임
#final_df = pd.DataFrame(columns=['종목코드', 'final_return'])
final_df = pd.DataFrame(['종목코드', '종가', '5평균', '10평균', '20평균', '60평균'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)

    # 20일 단위 날짜 인덱스 뽑아내기
    import datetime as dt
    date=[]
    for i in tqdm(list(train_close["20평균"].index + dt.timedelta(days=20))):
        arange=[]
        for j in range(20,0,-1):
            arange.append(i-dt.timedelta(days=j))
        date.append(arange)
    
    #####---------- 변수 2 _ 정배열 / 역배열 영향력 강화
    # 20일 단위 내 정배열 역배열 판단

    condition_1 = train_close['5평균'] > train_close['10평균'] > train_close['20평균'] > train_close['60평균']
    condition_2 = train_close['5평균'] < train_close['10평균'] < train_close['20평균'] < train_close['60평균']
    
    box = pd.DataFrame(columns=['날짜', '값'])
    
    for i in range(0,len(date),2):
        default = 0
        for j in range(len(date[i+1])):
            if train_close[train_close.index==date[i][j]] == condition_1 :
                default +=5
            elif train_close[train_close.index==date[i][j]] == condition_2:
                default -=5
            else:
                pass
        results = results.append({'종목코드': i, 'final_return': default}, ignore_index=True)
        

    #결과값
    final_df = pd.concat([final_df,train_close],axis=0)
    
final_df = final_df[['종목코드','종가','5평균','10평균','20평균','60평균']].dropna(axis=0)
final_df.to_csv("60without.csv",index=False)

  0%|          | 0/2000 [00:00<?, ?it/s]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
# 저장할 데이터 프레임
#final_df = pd.DataFrame(columns=['종목코드', 'final_return'])
final_df = pd.DataFrame(['종목코드', '기준날짜', '배열_가중치'])


# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

for code in tqdm(unique_codes):
#code = "A103840"
# # 이동평균선 간격 데이터 생성
    train_close = train[train['종목코드'] == code][['종목코드','일자', '종가']]
    train_close['일자'] = pd.to_datetime(train_close['일자'], format='%Y%m%d')
    train_close.set_index('일자', inplace=True)

    # 이동평균선 (5/10/20/60)
    train_close["5평균"] = train_close['종가'].rolling(window=5).mean()
    train_close["10평균"] = train_close['종가'].rolling(window=10).mean()
    train_close["20평균"] = train_close['종가'].rolling(window=20).mean()
    train_close["60평균"] = train_close['종가'].rolling(window=60).mean()

    #이동평균선 NaN 값 삭제
    train_close.dropna(axis=0,inplace=True)
    #####---------- 변수 2 _ 정배열 / 역배열 영향력 강화
    # 20일 단위 내 정배열 역배열 판단

    # 20일 단위 날짜 인덱스 뽑아내기
    import datetime as dt
    date=[]
    for i in list(train_close["20평균"].index + dt.timedelta(days=20)):
        arange=[]
        for j in range(20,0,-1):
            arange.append(i-dt.timedelta(days=j))
        date.append(arange)   

    #정배열 역배열 조건
    condition_1 = (train_close['5평균'] > train_close['10평균']) & (train_close['10평균'] >  train_close['20평균']) & (train_close['20평균'] > train_close['60평균'])
    condition_2 = (train_close['5평균'] < train_close['10평균']) & (train_close['10평균'] >  train_close['20평균']) & (train_close['20평균'] < train_close['60평균'])

    # 기준 날짜 넣기
    for i in range(0,len(date),2):
        default = 0
        for j in range(len(date[i+1])):
            if date[i][j] == train_close.index :
                if train_close["5평균"].iloc[i] > train_close["MA10_mean"].iloc[i] and train_close["MA10_mean"].iloc[i] > train_close["MA20_mean"].iloc[i] and train_close["MA20_mean"].iloc[i] > train_close["MA60_mean"].iloc[i]:

            if train_close[train_close.index==date[i][j]] == (train_close['5평균'] > train_close['10평균']) & (train_close['10평균'] >  train_close['20평균']) & (train_close['20평균'] > train_close['60평균']) :
                default +=5
            elif train_close[train_close.index==date[i][j]] == condition_2:
                default -=5
            else:
                pass
        final_df = final_df.append({'종목코드': code, '기준날짜': i, '배열_가중치': default}, ignore_index=True)      

    #결과값
    #final_df = pd.concat([final_df,train_close],axis=0)
    

   

  2%|▏         | 42/2000 [00:03<02:53, 11.28it/s]


KeyboardInterrupt: 

In [13]:
train_close[train_close.index==date]

ValueError: Lengths must match

In [3]:
final_df = pd.read_csv("./60without.csv")
final_df

,종목코드,종가,5평균,10평균,20평균,60평균
0,A060310,3315.0,3265.0,3409.5,3518.25,3236.583333
1,A060310,3335.0,3255.0,3375.5,3517.00,3243.500000
2,A060310,3380.0,3278.0,3354.5,3522.25,3251.500000
3,A060310,3465.0,3341.0,3332.5,3533.25,3260.916667
4,A060310,3445.0,3388.0,3325.0,3541.50,3269.166667
...,...,...,...,...,...,...
869995,A238490,6430.0,6540.0,6416.0,6440.50,6529.333333
869996,A238490,6320.0,6548.0,6399.0,6429.50,6529.833333
869997,A238490,6330.0,6502.0,6383.0,6421.00,6530.666667
869998,A238490,6330.0,6424.0,6393.0,6419.50,6530.000000


In [5]:
import datetime as dt

date=[]

for i in tqdm(list(final_df["20평균"].index + dt.timedelta(days=20))):
    date.append(i)
    arange=[]
    for j in range(20,0,-1):
        arange.append(i-dt.timedelta(days=j))
    date.append(arange)

date

TypeError: Concatenation operation is not implemented for NumPy arrays, use np.concatenate() instead. Please do not rely on this error; it may not be given on all Python implementations.

In [32]:
train_close

,종목코드,종가,5평균,10평균,20평균,정배열,역배열
일자,,,,,,,
2021-06-28,A238490,11750,11830.0,11675.0,11587.5,0,0
2021-06-29,A238490,11800,11830.0,11695.0,11602.5,0,0
2021-06-30,A238490,11750,11820.0,11725.0,11620.0,0,0
2021-07-01,A238490,11550,11740.0,11750.0,11620.0,0,0
2021-07-02,A238490,11500,11670.0,11760.0,11607.5,0,0
...,...,...,...,...,...,...,...
2023-05-23,A238490,6430,6540.0,6416.0,6440.5,0,0
2023-05-24,A238490,6320,6548.0,6399.0,6429.5,0,0
2023-05-25,A238490,6330,6502.0,6383.0,6421.0,0,0
